In [39]:
import dataclasses

class BipolarCircuit(object):
    def __or__(self, other):
        return Parallel(self, other)
    
    def __add__(self, other):
        return Serial(self, other)

class Combination(BipolarCircuit):
    def __init__(self, *args):
        self._components = args

    def __repr__(self):
        args = ', '.join(repr(item) for item in self._components)
        return f"{self.__class__.__name__}({args})"
    
    @property
    def bom(self):
        output = []
        for item in self._components:
            output.extend(item.bom)
        return output

class Serial(Combination):
    def impedance(self, omega):
        return sum([component.impedance(omega) for component in self._components])
    
class Parallel(Combination):
    def impedance(self, omega):
        return 1/(sum([1/component.impedance(omega) for component in self._components]))

@dataclasses.dataclass
class Device(BipolarCircuit):
    value : float

    @property
    def bom(self):
        return [self]
        
class Resistor(Device):
    def impedance(self, omega):
        return self.value
        
class Capacitor(Device):
    def impedance(self, omega):
        return 1/(1J*omega*self.value)
        
class Inductor(Device):
    def impedance(self, omega):
        return 1J*omega*self.value
        

In [42]:
my_circuit = Serial(Parallel(Resistor(10),Capacitor(1E-5),Inductor(15E-6)),
Resistor(5))
my_circuit = (Resistor(10)|Capacitor(1E-5)|Inductor(15E-6))+Resistor(5)
my_circuit#.impedance(1E5)
#Resistor(10)
my_circuit.bom

[Resistor(value=10),
 Capacitor(value=1e-05),
 Inductor(value=1.5e-05),
 Resistor(value=5)]

##### from matplotlib import pyplot as plt
import numpy as np

pulsation = np.logspace(4, 6, 500)
plt.semilogx(pulsation, np.abs(my_circuit.impedance(pulsation)))